In [29]:
import mat73
import pandas as pd
import numpy as np
import scipy.io
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
from sklearn.preprocessing import MinMaxScaler

### Data loading and preprocessing

In [30]:
import os

DATA_DIR = ''
if 'google.colab' not in str(get_ipython()) and "anuja" in os.environ.get('USER'):
    DATA_DIR = 'data/'
    

In [31]:
foof = pd.read_csv(DATA_DIR+"foof2features.csv")
foof = foof.rename(columns={"C1": "IDs" ,"C2": "Intercept", "C3": "Slope"})
foof

,IDs,Intercept,Slope
0,NDARAA075AMK,0.986272,1.825774
1,NDARAA112DMH,1.486650,1.888544
2,NDARAA117NEJ,1.593155,2.095749
3,NDARAA947ZG5,0.703331,1.724831
4,NDARAA948VFH,0.918020,1.749441
...,...,...,...
2037,NDARZN277NR6,1.351549,1.996940
2038,NDARZN578YDP,1.380795,2.036327
2039,NDARZN610GTY,0.339229,1.050644
2040,NDARZN677EYE,0.781225,1.470061


In [32]:
data = scipy.io.loadmat(DATA_DIR+'x.mat')  
df = pd.DataFrame(data['x'].reshape((data['x'].shape[0], -1)))

# sparsing
df = np.array(df).reshape(data['x'].shape)
df_sparsed = np.concatenate([np.expand_dims(df[:,:,i:i+2].mean(axis = 2), axis = 2) for i in range(0, data['x'].shape[2]-2, 2)], axis = 2)
df = pd.DataFrame(df_sparsed.reshape((df_sparsed.shape[0], -1)))

#scaling
norm = MinMaxScaler().fit(df)
df = norm.transform(df)
df = pd.DataFrame(df.reshape((df.shape[0], -1)))

columns = np.asarray([['Electrode %d - %d/2 Hz'%(i+1, j+1)] for i in range(df_sparsed.shape[1]) for j in range(df_sparsed.shape[2])])
df.columns = columns
df['IDs'] = foof['IDs']
df

,"(Electrode 1 - 1/2 Hz,)","(Electrode 1 - 2/2 Hz,)","(Electrode 1 - 3/2 Hz,)","(Electrode 1 - 4/2 Hz,)","(Electrode 1 - 5/2 Hz,)","(Electrode 1 - 6/2 Hz,)","(Electrode 1 - 7/2 Hz,)","(Electrode 1 - 8/2 Hz,)","(Electrode 1 - 9/2 Hz,)","(Electrode 1 - 10/2 Hz,)",...,"(Electrode 105 - 31/2 Hz,)","(Electrode 105 - 32/2 Hz,)","(Electrode 105 - 33/2 Hz,)","(Electrode 105 - 34/2 Hz,)","(Electrode 105 - 35/2 Hz,)","(Electrode 105 - 36/2 Hz,)","(Electrode 105 - 37/2 Hz,)","(Electrode 105 - 38/2 Hz,)","(Electrode 105 - 39/2 Hz,)",IDs
0,1.212006e-10,2.857691e-08,0.000006,0.001066,0.033290,0.143816,0.221608,0.467715,0.593581,0.401686,...,0.000007,0.000002,4.333844e-07,9.373567e-08,1.616373e-08,3.256580e-09,4.986456e-10,7.457518e-11,1.130004e-11,NDARAA075AMK
1,1.539866e-01,3.856645e-01,0.074284,0.216988,0.177267,0.161251,0.117761,0.132384,0.049898,0.004720,...,0.031636,0.009416,2.352137e-03,4.122513e-04,4.825197e-05,5.944433e-06,5.749150e-07,6.398086e-08,8.415637e-09,NDARAA112DMH
2,3.439263e-05,4.110931e-03,0.094118,0.367821,0.284890,0.164614,0.216934,0.333176,0.526883,0.511715,...,0.000311,0.000079,2.045964e-05,4.443460e-06,7.491343e-07,1.436770e-07,2.039047e-08,2.751924e-09,3.663748e-10,NDARAA117NEJ
3,2.832192e-01,3.613309e-01,0.378969,0.299813,0.247412,0.058588,0.015256,0.004514,0.003988,0.061608,...,0.158691,0.150551,1.580043e-01,1.518115e-01,1.233778e-01,1.242931e-01,1.009636e-01,8.498748e-02,7.690149e-02,NDARAA947ZG5
4,1.247342e-04,1.132987e-02,0.167700,0.368840,0.155464,0.104191,0.194459,0.288490,0.233210,0.125520,...,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,NDARAA948VFH
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2037,2.303407e-04,2.867255e-02,0.194945,0.081646,0.018422,0.076959,0.179779,0.300051,0.417245,0.459853,...,0.093399,0.058729,4.012462e-02,2.464919e-02,1.257993e-02,7.816617e-03,3.846395e-03,1.926406e-03,1.018633e-03,NDARZN277NR6
2038,1.385606e-02,2.124126e-02,0.032991,0.043456,0.048827,0.049759,0.074988,0.222227,0.444972,0.464653,...,0.016832,0.009602,5.877454e-03,3.190797e-03,1.418509e-03,7.564419e-04,3.146594e-04,1.311957e-04,5.687030e-05,NDARZN578YDP
2039,1.089647e-02,3.749309e-02,0.139003,0.289797,0.304728,0.196113,0.191950,0.342653,0.413368,0.344024,...,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,NDARZN610GTY
2040,6.411911e-03,2.687961e-02,0.095551,0.241121,0.434011,0.574767,0.694741,0.815939,0.678021,0.490314,...,0.059230,0.026090,1.169173e-02,4.410850e-03,1.294309e-03,4.329110e-04,1.073561e-04,2.536768e-05,5.924775e-06,NDARZN677EYE


In [33]:
beh = pd.read_csv(DATA_DIR+"behaviorals.csv")
print('Before:'+str(beh.shape))

most_common_disorders = ['Attention-Deficit/Hyperactivity Disorder', 'Anxiety Disorders', 'Specific Learning Disorder',
                         'Autism Spectrum Disorder', 'Disruptive', 'Communication Disorder',
                         'Depressive Disorders',  'No Diagnosis Given', 'Other Disorders']

category_columns = ['DX_' + str(i).zfill(2) + '_Cat' for i in range(1, 11)] +\
                   ['DX_' + str(i).zfill(2) + '_Sub' for i in range(1, 11)]

# removing patients with incomplete eval
initial_size = beh.shape[0]
beh = beh[beh.DX_01 != 'No Diagnosis Given: Incomplete Eval']
beh = beh.reset_index(drop=True)
new_size = beh.shape[0]

print('After:'+str(beh.shape))
print('Removing', initial_size - new_size,
      'patients as their evaluations was incomplete.')

Before:(3076, 177)
After:(2939, 177)
Removing 137 patients as their evaluations was incomplete.


In [34]:
no_diagnosis_given = 'No Diagnosis Given'

diagnoses_to_ids = {disorder: i for i, disorder in enumerate(most_common_disorders)}
diagnoses_to_ids

{'Attention-Deficit/Hyperactivity Disorder': 0,
 'Anxiety Disorders': 1,
 'Specific Learning Disorder': 2,
 'Autism Spectrum Disorder': 3,
 'Disruptive': 4,
 'Communication Disorder': 5,
 'Depressive Disorders': 6,
 'No Diagnosis Given': 7,
 'Other Disorders': 8}

In [35]:
def get_disorder(data, row, index):
    disorder = data.iloc[row][category_columns[index]]

    if disorder == 'Neurodevelopmental Disorders':
        disorder = data.iloc[row][category_columns[index + 10]]

    return disorder

order_of_disorders = []
for k in range(beh.shape[0]):
    i = 0
    disorder = get_disorder(beh, k, i)
    disorders_patient = []
    while not pd.isnull(disorder):
        if disorder in diagnoses_to_ids:
            if diagnoses_to_ids[disorder] not in disorders_patient:
                disorders_patient.append(diagnoses_to_ids[disorder])
        else:
            if diagnoses_to_ids['Other Disorders'] not in disorders_patient:
                disorders_patient.append(diagnoses_to_ids['Other Disorders'])
        i += 1
        if i == len(category_columns):
            break
        disorder = get_disorder(beh, k, i)

        
    order_of_disorders.append(disorders_patient)

In [36]:
other_disorders = []
no_diagnosis_given = []
for i in order_of_disorders:
    if 7 in i:
        no_diagnosis_given.append(1)
        i.remove(7)
    else:
        no_diagnosis_given.append(0)
    if 8 in i:
        other_disorders.append(1)
        i.remove(8)
    else:
        other_disorders.append(0)

In [37]:
max_len_order = np.max([len(x) for x in order_of_disorders])

# pad with a new token denoting the pad token
pad_token = len(most_common_disorders)
bod_token = len(most_common_disorders) + 1
eod_token = len(most_common_disorders) + 2

order_of_disorders = [[bod_token] + x + [eod_token] + [pad_token] * (max_len_order - len(x)) for x in order_of_disorders]

order_of_disorders = np.array(order_of_disorders)

classes = np.zeros((len(most_common_disorders),
                    beh.shape[0]), dtype=np.int32)

df_disorders = beh[category_columns]

for i, disorder in enumerate(most_common_disorders):
    mask = df_disorders.select_dtypes(include=[object]). \
        applymap(lambda x: disorder in x if pd.notnull(x) else False)

    disorder_df = df_disorders[mask.any(axis=1)]

    np.add.at(classes[i], disorder_df.index.values, 1)

behaviour_data_columns = beh.columns.values.astype(np.str)

columns_to_drop = behaviour_data_columns[
    np.flatnonzero(np.core.defchararray.find(behaviour_data_columns, 'DX') != -1)]

behaviour_data = beh.drop(columns=columns_to_drop)

for disorder, classification in zip(most_common_disorders, classes):
    behaviour_data[disorder] = classification

behaviour_data['order_diagnoses'] = list(order_of_disorders)

In [38]:
common_disorders = ['Attention-Deficit/Hyperactivity Disorder', 'Anxiety Disorders', 'Specific Learning Disorder',
                         'Autism Spectrum Disorder', 'Disruptive', 'Communication Disorder',
                         'Depressive Disorders']

labels=behaviour_data[["IDs"]+list(common_disorders)]
labels["Other Disorders"] = other_disorders
labels

/tmp/ipykernel_907427/226828375.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  labels["Other Disorders"] = other_disorders


,IDs,Attention-Deficit/Hyperactivity Disorder,Anxiety Disorders,Specific Learning Disorder,Autism Spectrum Disorder,Disruptive,Communication Disorder,Depressive Disorders,Other Disorders
0,NDARAA075AMK,0,0,0,0,0,0,0,0
1,NDARAA112DMH,1,0,0,0,1,0,0,1
2,NDARAA117NEJ,1,0,0,0,1,0,0,1
3,NDARAA306NT2,1,1,1,0,0,1,0,1
4,NDARAA504CRN,1,1,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...
2934,NDARZZ007YMP,0,0,0,1,0,0,0,1
2935,NDARZZ740MLM,1,0,0,0,0,0,0,0
2936,NDARZZ810LVF,0,0,0,1,0,1,0,1
2937,NDARZZ830JM7,0,0,0,1,0,0,0,1


In [39]:
df = pd.merge(df, foof, on='IDs', how='inner')
df = pd.merge(df, labels, on='IDs', how='inner')
df

,"(Electrode 1 - 1/2 Hz,)","(Electrode 1 - 2/2 Hz,)","(Electrode 1 - 3/2 Hz,)","(Electrode 1 - 4/2 Hz,)","(Electrode 1 - 5/2 Hz,)","(Electrode 1 - 6/2 Hz,)","(Electrode 1 - 7/2 Hz,)","(Electrode 1 - 8/2 Hz,)","(Electrode 1 - 9/2 Hz,)","(Electrode 1 - 10/2 Hz,)",...,Intercept,Slope,Attention-Deficit/Hyperactivity Disorder,Anxiety Disorders,Specific Learning Disorder,Autism Spectrum Disorder,Disruptive,Communication Disorder,Depressive Disorders,Other Disorders
0,1.212006e-10,2.857691e-08,0.000006,0.001066,0.033290,0.143816,0.221608,0.467715,0.593581,0.401686,...,0.986272,1.825774,0,0,0,0,0,0,0,0
1,1.539866e-01,3.856645e-01,0.074284,0.216988,0.177267,0.161251,0.117761,0.132384,0.049898,0.004720,...,1.486650,1.888544,1,0,0,0,1,0,0,1
2,3.439263e-05,4.110931e-03,0.094118,0.367821,0.284890,0.164614,0.216934,0.333176,0.526883,0.511715,...,1.593155,2.095749,1,0,0,0,1,0,0,1
3,2.832192e-01,3.613309e-01,0.378969,0.299813,0.247412,0.058588,0.015256,0.004514,0.003988,0.061608,...,0.703331,1.724831,1,0,1,1,0,0,0,1
4,1.247342e-04,1.132987e-02,0.167700,0.368840,0.155464,0.104191,0.194459,0.288490,0.233210,0.125520,...,0.918020,1.749441,1,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1916,1.556547e-05,2.900630e-04,0.003548,0.023882,0.088943,0.189421,0.293481,0.445277,0.644915,0.577096,...,0.168009,0.205704,0,1,0,0,0,0,0,0
1917,2.303407e-04,2.867255e-02,0.194945,0.081646,0.018422,0.076959,0.179779,0.300051,0.417245,0.459853,...,1.351549,1.996940,1,1,0,0,0,0,1,0
1918,1.089647e-02,3.749309e-02,0.139003,0.289797,0.304728,0.196113,0.191950,0.342653,0.413368,0.344024,...,0.339229,1.050644,0,0,0,0,0,0,0,1
1919,6.411911e-03,2.687961e-02,0.095551,0.241121,0.434011,0.574767,0.694741,0.815939,0.678021,0.490314,...,0.781225,1.470061,1,0,0,0,0,0,0,0


### Data Split

In [40]:
disorders_list = ['Attention-Deficit/Hyperactivity Disorder', 'Anxiety Disorders', 'Specific Learning Disorder',
                         'Autism Spectrum Disorder', 'Disruptive', 'Communication Disorder',
                         'Depressive Disorders', 'Other Disorders']
x = df[df.columns.difference(['IDs']+disorders_list)]
y = df[disorders_list]

# summarize dataset shape
print(x.shape, y.shape)

(1921, 4097) (1921, 8)


## FEATURE EXTRACTION WITH PCA

In [41]:
# dimensionality reduction
from sklearn.decomposition import PCA

pca = PCA(.95) # 95% variance retained
pca.fit(x)

# transform data
x_pca = pca.transform(x)
x_pca.shape



x_pca = x

In [42]:
train_features, test_features, train_labels, test_labels = train_test_split(x, y, test_size=0.25, shuffle=True)

In [43]:
#scaling features

# data normalization with sklearn
from sklearn.preprocessing import MinMaxScaler

# fit scaler on training data
norm = MinMaxScaler().fit(x)

# transform training data
x_norm = norm.transform(x)
x_norm = x


In [44]:
train_labels.sum(axis=0)

Attention-Deficit/Hyperactivity Disorder    875
Anxiety Disorders                           482
Specific Learning Disorder                  331
Autism Spectrum Disorder                    217
Disruptive                                  222
Communication Disorder                      209
Depressive Disorders                        154
Other Disorders                             578
dtype: int64

In [16]:
test_labels.sum(axis=0)

Attention-Deficit/Hyperactivity Disorder    290
Anxiety Disorders                           168
Specific Learning Disorder                  108
Autism Spectrum Disorder                     70
Disruptive                                   65
Communication Disorder                       83
Depressive Disorders                         45
Other Disorders                             187
dtype: int64

## FEATURE EXTRACTION WITH AUTOENCODER

In [19]:
import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Dense, LeakyReLU, BatchNormalization
from tensorflow.keras.utils import plot_model, to_categorical

n_inputs = train_features.shape[1]

visible = Input(shape=(n_inputs,))
e = Dense(n_inputs*2)(visible)
e = BatchNormalization()(e)
e = LeakyReLU()(e)
e = Dense(n_inputs)(e)
e = BatchNormalization()(e)
e = LeakyReLU()(e)
n_bottleneck = round(float(n_inputs) / 2.0)
bottleneck = Dense(n_bottleneck)(e)

d = Dense(n_inputs)(bottleneck)
d = BatchNormalization()(d)
d = LeakyReLU()(d)

d = Dense(n_inputs*2)(d)
d = BatchNormalization()(d)
d = LeakyReLU()(d)

output = Dense(8, activation='linear')(d)

model = Model(inputs=visible, outputs=output)
model.compile(optimizer='adam', loss='mse')
plot_model(model, 'autoencoder_compress.png', show_shapes=True)

history = model.fit(train_features, train_labels, epochs=50, batch_size=16, verbose=2)
encoder = Model(inputs=visible, outputs=bottleneck)

plot_model(encoder, 'encoder_compress.png', show_shapes=True)
encoder.save('autoencoder.h5')

Failed to import pydot. You must install pydot and graphviz for `pydotprint` to work.
Epoch 1/50


2021-11-25 22:52:42.514731: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10


90/90 - 2s - loss: 8.0477
Epoch 2/50
90/90 - 2s - loss: 0.2293
Epoch 3/50
90/90 - 2s - loss: 0.2331
Epoch 4/50
90/90 - 2s - loss: 0.2468
Epoch 5/50
90/90 - 2s - loss: 0.2478
Epoch 6/50
90/90 - 2s - loss: 0.2730
Epoch 7/50
90/90 - 2s - loss: 0.2857
Epoch 8/50
90/90 - 2s - loss: 0.2685
Epoch 9/50
90/90 - 2s - loss: 0.2675
Epoch 10/50
90/90 - 2s - loss: 0.2953
Epoch 11/50
90/90 - 2s - loss: 0.2988
Epoch 12/50
90/90 - 2s - loss: 0.2637
Epoch 13/50
90/90 - 2s - loss: 0.2797
Epoch 14/50
90/90 - 2s - loss: 0.2756
Epoch 15/50
90/90 - 2s - loss: 0.3044
Epoch 16/50
90/90 - 2s - loss: 0.3169
Epoch 17/50
90/90 - 2s - loss: 0.3140
Epoch 18/50
90/90 - 2s - loss: 0.3291
Epoch 19/50
90/90 - 2s - loss: 0.3381
Epoch 20/50
90/90 - 2s - loss: 0.2861
Epoch 21/50
90/90 - 2s - loss: 0.2923
Epoch 22/50
90/90 - 2s - loss: 0.3024
Epoch 23/50
90/90 - 2s - loss: 0.2685
Epoch 24/50
90/90 - 2s - loss: 0.2852
Epoch 25/50
90/90 - 2s - loss: 0.3235
Epoch 26/50
90/90 - 2s - loss: 0.3431
Epoch 27/50
90/90 - 2s - loss: 0

In [20]:
# encode the data
encoder= load_model('autoencoder.h5', compile=False)

train_features = encoder.predict(train_features)
test_features = encoder.predict(test_features)

### Metrics

In [45]:
from sklearn.metrics import hamming_loss, accuracy_score
import sklearn.metrics as skm
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import numpy as np

def brier_multi(targets, probs):
    return np.mean(np.sum((probs - targets)**2, axis=1))

def evaluate(y_test, y_pred_prob, brier=True):
    y_pred = y_pred_prob.round()
    print("Accuracy:", accuracy_score(y_test, y_pred))
    print("Hamming Loss:", hamming_loss(y_test, y_pred))
    if brier:
        print("Brier Score:", brier_multi(y_test, y_pred_prob))
    print("Classification Report:\n", skm.classification_report(y_test,y_pred, zero_division=1))
    print("Confusion matrix:\n", skm.multilabel_confusion_matrix(y_test, y_pred))

## Models

### Multi Output Classifier

In [46]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC as svm
from sklearn.multioutput import MultiOutputClassifier

forest = RandomForestClassifier(random_state=1)
lg = LogisticRegression()
svm = svm()
models = [lg, forest, svm]

for model in models:

    multi_output_model = MultiOutputClassifier(model, n_jobs=-1)
    multi_output_model.fit(train_features, train_labels)
    predicted_labels = multi_output_model.predict(test_features)
    print(str(model)+':')
    evaluate(test_labels, predicted_labels, brier=False)

/home/ubuntu/anaconda3/envs/ai4halth/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/ubuntu/anaconda3/envs/ai4halth/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/lin

LogisticRegression():
Accuracy: 0.06860706860706861
Hamming Loss: 0.28534303534303534
Classification Report:
               precision    recall  f1-score   support

           0       0.61      0.66      0.63       291
           1       0.37      0.26      0.30       158
           2       0.20      0.12      0.15       108
           3       0.21      0.08      0.12        75
           4       0.12      0.05      0.07        75
           5       0.10      0.03      0.05        63
           6       0.42      0.12      0.19        40
           7       0.36      0.34      0.35       179

   micro avg       0.43      0.33      0.37       989
   macro avg       0.30      0.21      0.23       989
weighted avg       0.37      0.33      0.34       989
 samples avg       0.54      0.43      0.33       989

Confusion matrix:
 [[[ 65 125]
  [ 99 192]]

 [[252  71]
  [117  41]]

 [[321  52]
  [ 95  13]]

 [[384  22]
  [ 69   6]]

 [[377  29]
  [ 71   4]]

 [[399  19]
  [ 61   2]]

 [[434   7

### MLP

In [47]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

def get_mlp(n_inputs, n_outputs):
    model = Sequential()
    model.add(Dense(20, input_dim=n_inputs, kernel_initializer='he_uniform', activation='relu'))
    model.add(Dense(n_outputs, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam')
    return model

n_inputs, n_outputs = train_features.shape[1],train_labels.shape[1]
mlp = get_mlp(n_inputs, n_outputs)
mlp.fit(train_features, train_labels, verbose=1, epochs=100)

Epoch 1/100
45/45 [==============================] - 0s 1ms/step - loss: 0.5466
Epoch 2/100
45/45 [==============================] - 0s 1ms/step - loss: 0.5235
Epoch 3/100
45/45 [==============================] - 0s 1ms/step - loss: 0.5197
Epoch 4/100
45/45 [==============================] - 0s 1ms/step - loss: 0.5146
Epoch 5/100
45/45 [==============================] - 0s 1ms/step - loss: 0.5100
Epoch 6/100
45/45 [==============================] - 0s 1ms/step - loss: 0.5072
Epoch 7/100
45/45 [==============================] - 0s 1ms/step - loss: 0.5023
Epoch 8/100
45/45 [==============================] - 0s 1ms/step - loss: 0.4980
Epoch 9/100
45/45 [==============================] - 0s 1ms/step - loss: 0.4932
Epoch 10/100
45/45 [==============================] - 0s 1ms/step - loss: 0.4886
Epoch 11/100
45/45 [==============================] - 0s 1ms/step - loss: 0.4856
Epoch 12/100
45/45 [==============================] - 0s 1ms/step - loss: 0.4796
Epoch 13/100
45/45 [=================

In [48]:
predicted_labels_mlp = mlp.predict(test_features)
evaluate(test_labels, predicted_labels_mlp)

Accuracy: 0.06444906444906445
Hamming Loss: 0.28326403326403327
Brier Score: 1.7176272807338944
Classification Report:
               precision    recall  f1-score   support

           0       0.61      0.55      0.58       291
           1       0.39      0.32      0.35       158
           2       0.26      0.14      0.18       108
           3       0.24      0.05      0.09        75
           4       0.28      0.11      0.15        75
           5       0.06      0.03      0.04        63
           6       0.23      0.07      0.11        40
           7       0.40      0.46      0.42       179

   micro avg       0.43      0.33      0.37       989
   macro avg       0.31      0.22      0.24       989
weighted avg       0.39      0.33      0.35       989
 samples avg       0.55      0.42      0.32       989

Confusion matrix:
 [[[ 89 101]
  [131 160]]

 [[243  80]
  [107  51]]

 [[331  42]
  [ 93  15]]

 [[393  13]
  [ 71   4]]

 [[385  21]
  [ 67   8]]

 [[384  34]
  [ 61   2]]



### Binary Relevance
ignores the possible correlations between class labels

In [49]:
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.naive_bayes import GaussianNB

classifier = BinaryRelevance(GaussianNB())
classifier.fit(test_features, test_labels)

BinaryRelevance(classifier=GaussianNB(), require_dense=[True, True])

In [50]:
predicted_labels_br = classifier.predict_proba(test_features)
evaluate(test_labels, predicted_labels_br.todense())

Accuracy: 0.04781704781704782
Hamming Loss: 0.4041060291060291
Brier Score: 3.2221146411774755
Classification Report:
               precision    recall  f1-score   support

           0       0.77      0.52      0.62       291
           1       0.40      0.87      0.55       158
           2       0.32      0.86      0.47       108
           3       0.37      0.40      0.38        75
           4       0.20      0.89      0.33        75
           5       0.22      0.73      0.34        63
           6       0.15      0.90      0.26        40
           7       0.51      0.66      0.58       179

   micro avg       0.35      0.69      0.47       989
   macro avg       0.37      0.73      0.44       989
weighted avg       0.48      0.69      0.51       989
 samples avg       0.38      0.71      0.43       989

Confusion matrix:
 [[[144  46]
  [140 151]]

 [[114 209]
  [ 20 138]]

 [[176 197]
  [ 15  93]]

 [[355  51]
  [ 45  30]]

 [[139 267]
  [  8  67]]

 [[255 163]
  [ 17  46]]

 

### Classfier Chains

In [51]:
from skmultilearn.problem_transform import ClassifierChain
from sklearn.linear_model import LogisticRegression

classifier = ClassifierChain(LogisticRegression())
classifier.fit(train_features, train_labels)
# we should optimise this a little

/home/ubuntu/anaconda3/envs/ai4halth/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/ubuntu/anaconda3/envs/ai4halth/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/lin

ClassifierChain(classifier=LogisticRegression(), require_dense=[True, True])

In [52]:
predicted_labels_cc = classifier.predict_proba(test_features)
evaluate(test_labels, predicted_labels_cc.todense())

Accuracy: 0.07276507276507277
Hamming Loss: 0.28326403326403327
Brier Score: 1.6334844077705128
Classification Report:
               precision    recall  f1-score   support

           0       0.61      0.66      0.63       291
           1       0.36      0.27      0.31       158
           2       0.22      0.13      0.16       108
           3       0.20      0.09      0.13        75
           4       0.21      0.11      0.14        75
           5       0.05      0.02      0.02        63
           6       0.41      0.17      0.25        40
           7       0.36      0.25      0.30       179

   micro avg       0.43      0.32      0.37       989
   macro avg       0.30      0.21      0.24       989
weighted avg       0.38      0.32      0.34       989
 samples avg       0.57      0.42      0.32       989

Confusion matrix:
 [[[ 65 125]
  [ 99 192]]

 [[249  74]
  [116  42]]

 [[323  50]
  [ 94  14]]

 [[378  28]
  [ 68   7]]

 [[375  31]
  [ 67   8]]

 [[399  19]
  [ 62   1]]



### Label Powerset
takes correlations into account!

In [53]:
from skmultilearn.problem_transform import LabelPowerset

classifier = LabelPowerset(LogisticRegression())
classifier.fit(train_features, train_labels)

/home/ubuntu/anaconda3/envs/ai4halth/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LabelPowerset(classifier=LogisticRegression(), require_dense=[True, True])

In [54]:
predicted_labels_lp = classifier.predict_proba(test_features)
evaluate(test_labels, predicted_labels_lp.todense())

Accuracy: 0.10810810810810811
Hamming Loss: 0.24896049896049896
Brier Score: 1.4215643591297917
Classification Report:
               precision    recall  f1-score   support

           0       0.62      0.78      0.69       291
           1       0.39      0.14      0.20       158
           2       0.20      0.02      0.03       108
           3       0.25      0.04      0.07        75
           4       0.20      0.04      0.07        75
           5       0.00      0.00      0.00        63
           6       0.00      0.00      0.00        40
           7       0.41      0.22      0.29       179

   micro avg       0.53      0.30      0.38       989
   macro avg       0.26      0.16      0.17       989
weighted avg       0.37      0.30      0.30       989
 samples avg       0.64      0.41      0.35       989

Confusion matrix:
 [[[ 50 140]
  [ 64 227]]

 [[288  35]
  [136  22]]

 [[365   8]
  [106   2]]

 [[397   9]
  [ 72   3]]

 [[394  12]
  [ 72   3]]

 [[415   3]
  [ 63   0]]



### Multi Label KNN

In [55]:
from skmultilearn.adapt import MLkNN
from scipy.sparse import csr_matrix, lil_matrix

mlknn = MLkNN(k=10)

x_train = lil_matrix(train_features).toarray()
y_train = lil_matrix(train_labels).toarray()
x_test = lil_matrix(test_features).toarray()

mlknn.fit(x_train, y_train)

/home/ubuntu/anaconda3/envs/ai4halth/lib/python3.8/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass n_neighbors=10 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


MLkNN()

In [56]:
predicted_labels_mlknn = mlknn.predict_proba(x_test)
evaluate(test_labels, predicted_labels_mlknn.todense())

Accuracy: 0.10602910602910603
Hamming Loss: 0.257016632016632
Brier Score: 1.8118628649328041
Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.00      0.00       291
           1       1.00      0.00      0.00       158
           2       1.00      0.00      0.00       108
           3       1.00      0.00      0.00        75
           4       1.00      0.00      0.00        75
           5       1.00      0.00      0.00        63
           6       1.00      0.00      0.00        40
           7       1.00      0.00      0.00       179

   micro avg       1.00      0.00      0.00       989
   macro avg       1.00      0.00      0.00       989
weighted avg       1.00      0.00      0.00       989
 samples avg       1.00      0.11      0.11       989

Confusion matrix:
 [[[190   0]
  [291   0]]

 [[323   0]
  [158   0]]

 [[373   0]
  [108   0]]

 [[406   0]
  [ 75   0]]

 [[406   0]
  [ 75   0]]

 [[418   0]
  [ 63   0]]

 [